<a href="https://colab.research.google.com/github/starirene9/DeepLearningAssignment/blob/main/4%E1%84%8C%E1%85%AE%E1%84%8E%E1%85%A1_%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

import torchvision.transforms as T
from torchvision.datasets import MNIST

In [ ]:
LR = 0.1
EPOCH = 10 # 전체 데이터셋을 한 번 학습하는 횟수
BATCH_SIZE = 64 # 한 번에 학습하는 데이터 샘플 개수
# 배치 사이즈 늘릴 것~! GPU 최대한 늘이기 위해서
# 배치 사이즈를 무조건 늘리면 학습 시간이 오래 걸림

In [ ]:
train = MNIST('./', train=True, download=True, transform=T.ToTensor()) # train=True 데이터로 모델을 학습
test = MNIST('./', train=False, download=False, transform=T.ToTensor()) # train=False 데이터로 모델을 평가

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.44MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.52MB/s]


In [ ]:
# 데이터를 실제 학습에 사용할 수 있는 배치 사이즈 만큼 읽어서 제공 : 데이터를 미리 읽어 놓고 바로바로 보내줌

train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True) # shuffle : 데이터를 섞음
test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
#첫 번째 배치만 가져오는 것은 훈련을 시작하기 전에 데이터와 모델이 올바르게 설정되었는지 빠르게 확인하기 위해서
img, target = next(iter(train_loader)) # 첫번째 배치 반환
print(img.size())
print(target.size())  # 타겟(정답) 텐서의 크기 출력

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [ ]:
model = torch.nn.Sequential(
    ### Make a model ###
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),   # 활성화 함수 (비선형성 추가)
    torch.nn.Linear(512, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10),
    # 마지막에 softmax가 들어가므로 또 한번 넣지 않아도 됨
).cuda()

# 손실 함수 정의 (CrossEntropyLoss는 softmax를 내부적으로 포함)
loss_function = torch.nn.CrossEntropyLoss().cuda()

"""
🔹 CrossEntropyLoss의 특징
	1.	Softmax가 포함됨 → logits(출력값)만 입력하면 됨.
	2.	다중 클래스 분류에 적합 → target은 정답 클래스의 인덱스(정수) 형태로 제공해야 함.
	3.	확률이 정답에 가까울수록 손실이 작아짐
	4.	예측이 틀릴수록 손실이 커짐
"""

# 옵티마이저 설정 (SGD: 확률적 경사 하강법, 학습률은 LR)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
# 모델의 모든 가중치와 편향(bias)을 업데이트하기 위해 model.parameters()를 옵티마이저에 전달
# lr (learning rate) 는 가중치를 업데이트하는 속도를 결정하는 값

In [ ]:
model.train()

for epoch in range(EPOCH): # 전체 데이터를 몇번 볼것인가
    for img, target in train_loader:
        optimizer.zero_grad()  # 기존의 그래디언트 초기화 (중요)
        img = img.flatten(1).cuda() # 2D 이미지를 1D 벡터(784 차원)로 변환 후 GPU로 이동
        #Fully Connected Layer (Linear Layer) 를 사용할 때는 1D 벡터 형태로 변환
        target = target.cuda()

        # 모델을 통해 예측값(logit) 계산
        logit = model(img)
        loss = loss_function(logit, target)

        loss.backward() # 역전파(Backpropagation)를 수행하여 기울기 계산
        optimizer.step() # 옵티마이저가 가중치를 업데이트

        """
        ✔ loss.backward()는 손실을 기반으로 가중치의 기울기를 계산하는 과정
        ✔ 기울기가 없으면 가중치를 어떻게 업데이트해야 할지 알 수 없기 때문에 학습 불가능
        ✔ 역전파 후 optimizer.step()을 호출하면 기울기를 사용하여 가중치를 업데이트
        """

    if epoch % 5 == 0:  # 5 에포크마다 손실 값 출력
        print(f'{epoch + 1} epoch: {loss.item()}')

1 epoch: 0.37449732422828674
6 epoch: 0.04162312299013138


In [ ]:
# 학습된 모델을 사용하여 테스트 데이터의 정확도를 평가하는 과정

"""
드롭아웃(Dropout) 은 신경망 학습 시 일부 뉴런을 랜덤하게 비활성화(0으로 만듦)하여 과적합(Overfitting)을 방지하는 기법입니다.
즉, 학습할 때마다 일부 뉴런을 꺼서 모델이 특정 패턴에 과도하게 의존하지 않도록 유도하는 역할을 합니다.
"""
model.eval() # 신경망에서 드롭아웃(Dropout)과 배치 정규화(Batch Normalization) 같은 레이어를 비활성화

total = 0  # 전체 테스트 데이터 개수를 저장할 변수 (모든 샘플 개수)
correct = 0  # 모델이 정답을 맞힌 개수를 저장할 변수


# 평가 모드에서는 자동 미분 계산을 하지 않도록 설정 (메모리 절약)
# 테스트 과정에서는 기울기(Gradient) 계산이 필요 없음
with torch.no_grad():
    for imgs, targets in test_loader:
        imgs = imgs.flatten(1).cuda()  # 입력 데이터 변환 후 GPU 이동
        targets = targets.cuda()

        logits = model(imgs) #변환된 입력 데이터를 모델에 넣어 출력(logits) 계산
        _, preds = torch.max(logits.data, 1)  # 가장 높은 확률을 가진 클래스 인덱스 반환, 1은 열 방향
        """
        •	logits에서 가장 큰 값의 인덱스를 반환
	      •	1은 두 번째 차원(클래스 차원)에서 최대값을 찾는다는 의미

        logits = torch.tensor([[2.5, 1.2, 0.3],  # 첫 번째 샘플
                       [0.1, 3.6, 1.8],  # 두 번째 샘플
                       [1.2, 0.7, 4.5]]) # 세 번째 샘플

        values, indices = torch.max(logits, 1)  # 각 행(샘플)에서 최대값 찾기

        print("최댓값:", values)
        print("최댓값의 인덱스:", indices)

        최댓값: tensor([2.5, 3.6, 4.5])
        최댓값의 인덱스: tensor([0, 1, 2])

        •	첫 번째 반환값(최댓값)은 _에 저장 → 사용하지 않겠다는 의미
        •	두 번째 반환값(최댓값의 인덱스)은 preds에 저장 → 예측 클래스 번호
        """

        total += targets.size(0) # 총 샘플 개수 업데이트 : size(0)은 첫 번째 차원(배치 크기)을 가져옴
        correct += (preds == targets).sum().item()  #  최종적으로 correct에는 모든 테스트 데이터에서 맞춘 개수가 저장됨

        """

          # 가정: 테스트 데이터에서 4개의 샘플(batch)
          preds   = torch.tensor([0, 2, 1, 3])  # 모델 예측값
          targets = torch.tensor([0, 1, 1, 3])  # 실제 정답

          correct = 0  # 맞춘 개수 초기화
          correct += (preds == targets).sum().item()

          print(f'정확히 맞춘 개수: {correct}')
        """

# 정확도(%) 출력
print(100 * correct / total)

97.91
